# PASO A PASO
1. Importar librerías

In [1]:
import psycopg2
import psycopg2.extras
#connection = None
#cursor = None

2. Generar conexión

In [2]:
connection = psycopg2.connect(
        host= 'localhost',
        dbname = "proyectoX",
        user= "postgres" ,
        password = 'PRUEBA123',
        port = 5433)

3. Generar consultas python
- ¿Total de ventas que se hicieron en mayo y junio para los años 2021 y 2022 en la tabla customers?
## Solución con Python

In [3]:
#Selecciono toda mi tabla para después ponerla en un df
Statement = "SELECT * FROM customers"

In [4]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor.execute(Statement)
df = cursor.fetchall()
print(type(df))

<class 'list'>


In [5]:
#Creo mi df y le cambio el nombre a las columnas
import pandas as pd
df = pd.DataFrame(df)
df.columns =["id","first_name","Last_name","email","gender","product","price","quantity","fecha"]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101 entries, 0 to 100
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id          101 non-null    int64         
 1   first_name  101 non-null    object        
 2   Last_name   101 non-null    object        
 3   email       101 non-null    object        
 4   gender      101 non-null    object        
 5   product     101 non-null    object        
 6   price       101 non-null    object        
 7   quantity    101 non-null    int64         
 8   fecha       101 non-null    datetime64[ns]
dtypes: datetime64[ns](1), int64(2), object(6)
memory usage: 7.2+ KB


In [6]:
#paso a float los datos de la columna price que estaban en moneda
df["price"] = [i.replace(",",".") for i in df["price"]]
df["price"] = [i.replace("$","") for i in df["price"]]
df["price"] = [float(i) for i in df["price"]]



In [7]:
# paso a enteros los datos de la col quantity
df["quantity"] = [int(i) for i in df["quantity"]]

In [8]:
# creo una nueva col de ventas a partir de la columna quantity * price
df["ventas"] = df["quantity"]* df["price"]

In [9]:
# creo un nuevo df llamado filtered_df en donde filtro por el mes 5,6 y el año 2021 y 2022 
import datetime

df["mes"] = df["fecha"].dt.month
df["ano"] = df["fecha"].dt.year
filtered_df = df[df["mes"].isin([5,6]) &df["ano"].isin([2021,2022])]
filtered_df.head(10)

,id,first_name,Last_name,email,gender,product,price,quantity,fecha,ventas,mes,ano
0,2,Magda,Ascough,mascough1@noaa.gov,Female,Longos - Chicken Cordon Bleu,721.0,2,2022-05-05,1442.0,5,2022
1,3,Alexander,Marchington,amarchington2@alibaba.com,Male,Kellogs All Bran Bars,432.0,3,2022-05-07,1296.0,5,2022
8,10,Jacquette,Drewett,jdrewett9@weebly.com,Female,Cheese - Bakers Cream Cheese,998.0,10,2022-05-23,9980.0,5,2022
23,25,Llewellyn,Janes,ljaneso@purevolume.com,Male,"Vermouth - Sweet, Cinzano",290.0,25,2022-06-23,7250.0,6,2022
44,46,Robin,Jagiello,rjagiello19@ed.gov,Bigender,"Beans - Kidney, Red Dry",856.0,46,2022-06-26,39376.0,6,2022
45,47,Maritsa,Kettlewell,mkettlewell1a@discovery.com,Female,Cucumber - Pickling Ontario,843.0,47,2022-06-24,39621.0,6,2022
50,52,Roxana,Ternent,rternent1f@home.pl,Female,Breadfruit,347.0,52,2022-06-14,18044.0,6,2022
51,53,Cecile,Boag,cboag1g@tuttocitta.it,Female,"Squash - Pattypan, Yellow",500.0,53,2022-05-16,26500.0,5,2022
54,56,Stevy,Dietsche,sdietsche1j@bloglines.com,Male,"Arctic Char - Fresh, Whole",239.0,56,2022-05-09,13384.0,5,2022
70,72,Tish,Biskupiak,tbiskupiak1z@about.com,Female,Assorted Desserts,394.0,72,2022-06-10,28368.0,6,2022


In [10]:
# con el df filtered_df agrupo los valores de mes y ano segun las sum de las ventas
answer = filtered_df["ventas"].groupby([filtered_df["mes"],filtered_df["ano"]]).sum()
answer

mes  ano 
5    2022     52602.0
6    2022    148334.0
Name: ventas, dtype: float64

2. ¿Cuales son los datos duplicados en la tabla customers?
## Solución con Python

In [11]:
# si te das cuenta df.duplicates me arroja una serie booleana False or True
df_duplicates = df.duplicated()
df_duplicates.info()

<class 'pandas.core.series.Series'>
RangeIndex: 101 entries, 0 to 100
Series name: None
Non-Null Count  Dtype
--------------  -----
101 non-null    bool 
dtypes: bool(1)
memory usage: 229.0 bytes


In [12]:
# por eso nos toca filtrar por el df primero
df_duplicates = df[df.duplicated()]
df_duplicates

,id,first_name,Last_name,email,gender,product,price,quantity,fecha,ventas,mes,ano


In [13]:
# con subset especificamos las columnas que queremos verificar si están duplicadas
df_duplicates = df[df.duplicated(subset=["first_name","Last_name","email"])]
df_duplicates

,id,first_name,Last_name,email,gender,product,price,quantity,fecha,ventas,mes,ano
100,103,Whit,Hayball,whayball0@spotify.com,Male,"Wine-Red,Antinori Santa",899.0,1,2022-08-09,899.0,8,2022


3.  ¿Muestreme las incoherencias de las columnas product entre las tablas customers y product?

In [14]:
# Vamos a crear otro df llamado df_2 pero esta vez de la tabla product
Statemet_2 = "SELECT * FROM product"

In [15]:
cursor = connection.cursor(cursor_factory=psycopg2.extras.DictCursor)
cursor.execute(Statemet_2)
df_2 = cursor.fetchall()
df_2 = pd.DataFrame(df_2)
df_2.columns = ["product_id","product","costxunit","company","sector"]

In [16]:
# eliminamos los posibles espacios vacios de ambas columnas producto de ambas tablas
df["product"] = [i.strip() for i in df["product"]]
df_2["product"] = [i.strip() for i in df_2["product"]]

In [17]:
# verificamos si los producto de la tabla df estan en la tabla df_2 (arroja valores booleanos)
df["check"] = df["product"].isin(df_2["product"])
# si los valores booleanos de la columna check (creada anterior) son True, traigame de df la columna product
print(df.loc[df["check"] == True,"product"])

48              Ecolab Silver Fusion
64    Flour - Masa De Harina Mexican
66               Stock - Veal, Brown
69                  Salsify, Organic
Name: product, dtype: object
